In [3]:
%matplotlib inline
import json 
import matplotlib.pyplot as plt
import os
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import keras 

# import numpy
# import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf


from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix


from keras import backend as K

Using TensorFlow backend.
/home/omid/.conda/envs/iot_new/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Main configs

In [4]:
loadFromMerged=True
loadFromIndexes= False
Mapper='S'
IgnoreEmpty= True
FoldID ="1"
Epoch_count=100
Batch_size=5000

# Load the data the old way


In [5]:
# x_data= [] 
# y_data= [] 


# with open( '../files/txt/seq_mapping_large.txt' ) as f:
#     x_data =   f.readlines()

# with open( '../files/txt/command_mapping_large.txt' ) as f:
#     y_data = f.readlines()
    
    
# x_data =[ np.array([ int(y) for y in x.strip().split( ' ') ])   for x in  x_data ] 
# y_data =[ x.strip().split(' ') for x in  y_data ] 


#  Load The Data The New Way

In [6]:
#  mapps the input records to a integer array for the input
def mapping_x( inp, includeDirection = False , TrimAt= 15 ):
    if includeDirection:
        return np.array([ int(x["packet_length"]) * (1 if x['packet_source']=='hub' else -1)  for x in inp ][:15])
    else:
        return np.array([ int(x["packet_length"])  for x in inp ][:15])

In [7]:
def mapping_y_service(inp):
    return np.array(  list(set([x["event"] for x in inp])) if (len(inp )>0) else ["none"] )

def mapping_y_service_event(inp):
    return np.array(  list(set([ "%s-%s"%( x["event"] ,x["val"] ) for x in inp])) if (len(inp )>0) else ["none"] )

def mapping_y_device_service(inp):
    return np.array(  list(set([ "%s & %s"%( x["device"] ,x["event"] ) for x in inp])) if (len(inp )>0) else ["none"] )

def mapping_y_full(inp):
    return np.array(  list(set([ "%s & %s & %s"%( x["device"] ,x["event"], x['val'] ) for x in inp])) if (len(inp )>0) else ["none"] )


In [8]:
# this cleans the data removing emply nodes and turning the nodes into sarrays by calling the mapping function 
def clean_data( x_data, y_data , removeempty=True, Mapping='S'):
    cleans = [] 
    cleans = (sorted([ x for x in y_data if (removeempty and len(y_data[x]) > 0) or not removeempty  ] ))
    
    ret_x  = [x_data[x] for x in cleans]
    ret_y  = [y_data[x] for x in cleans] 
    
    print( len(y_data), len(cleans) )
    
    ret_x  = [ mapping_x(x) for x in ret_x ] 
    if Mapping=='S':
        ret_y  = [ mapping_y_service(y) for y in ret_y ]
    elif Mapping=='SE':
        ret_y  = [ mapping_y_service_event(y) for y in ret_y ]
    elif Mapping=='DS':
        ret_y  = [ mapping_y_device_service(y) for y in ret_y ]
    elif Mapping=='F':
        ret_y  = [ mapping_y_full(y) for y in ret_y ]
    return ret_x, ret_y

In [9]:
x= []
y= []

x_test = []
y_test = []

x_train = {}
y_train = {}

test_names = []

if loadFromMerged:
    print("in load from merged")
    with open(  '../files/train/merged/hub_segments_2.json'  ) as f:
        y_data = json.load(f)

    with open(  '../files/train/merged/pcap_segments_2.json'  ) as f:
        x_data = json.load(f)
  
    if len( y_data ) != len(x_data) :
        print( pick )
        
    print(len(y_data))
    #     continue
    if loadFromIndexes:
        print("load from indexes")
        with open("../files/train/merged/items_2_test-train_indexes.json")  as f:
            index_info = json.load(f)


        for i in index_info[FoldID]["test"]:
            x_test[str(i)]=( x_data[str(i)] )
            y_test[str(i)]=(  y_data[str(i)] )

        for i in index_info[FoldID]["train"]:
            x_train[str(i)]=(  x_data[str(i)] )
            y_train[str(i)]=(  y_data[str(i)] )
        
        x_test_t,y_test_t= clean_data( x_test, y_test, IgnoreEmpty , Mapping=Mapper)
        x_test.append(x_test_t)
        y_test.append(y_test_t)
        

    else :
        print("loading from test files")
        test_files = sorted(glob.glob( '../files/train/test/test_homes/pcap_segments_2/*.json' ))
        print( "found files : " , len(test_files) )
        for pick  in test_files:
            fname  = os.path.basename(pick)
            test_names.append( fname )
            with open( os.path.join( '../files/train/test/test_homes/hub_segments_2/', fname) ) as f:
                y_data_test = json.load(f)

            with open( os.path.join('../files/train/test/test_homes/pcap_segments_2/', fname) ) as f:
                x_data_test = json.load(f)
        
            
            t_x,t_y= clean_data( x_data_test, y_data_test, False , Mapping=Mapper )
           
            x_test.append(t_x)
            y_test.append(t_y)
            
#     x_test = x_data[ index_info["1"]["test"]  ]
#     y_test = y_data[ index_info["1"]["test"]  ]
    
#     x_train = x_data[ index_info["1"]["train"]  ]
#     y_train = y_data[ index_info["1"]["train"]  ]
    
    
    
    x_train,y_train= clean_data( x_data, y_data, IgnoreEmpty , Mapping=Mapper )

#     x.extend(t_x)
#     y.extend(t_y)
else:
    for pick in sorted(glob.glob( '../files/train/hub_segments/*.json' )):
        fname  = os.path.basename(pick)
        test_names.append( fname )
        with open( os.path.join( '../files/train/hub_segments/', fname) ) as f:
            y_data = json.load(f)

        with open( os.path.join('../files/train/pcap_segments/', fname) ) as f:
            x_data = json.load(f)

        if len( y_data ) != len(x_data) :
            print( pick )
            continue

        t_x,t_y= clean_data( x_data, y_data, True )

        x.extend( t_x)
        y.extend(t_y)

x= np.array(x)
y= np.array(y)

len(x_train), len(y_test)

in load from merged
31273
loading from test files
found files :  9
27720 27720
17593 17593
5722 5722
32069 32069
19968 19968
6404 6404
22974 22974
14568 14568
4970 4970
31273 8795


(8795, 9)

In [39]:
len(y_data)

31273

In [40]:
list(np.unique(  np.concatenate( y_train  )))

['acceleration',
 'activity',
 'battery',
 'button',
 'colorTemperature',
 'contact',
 'level',
 'lock',
 'motion',
 'on/off',
 'ping',
 'raw',
 'read_attr_-_raw',
 'status',
 'switch',
 'temperature',
 'threeAxis',
 'water']

In [41]:
classes = sorted(list(np.unique(  np.concatenate( y_train  ))))
[ (i , classes[i]) for i in range( len(classes) ) ]




[(0, 'acceleration'),
 (1, 'activity'),
 (2, 'battery'),
 (3, 'button'),
 (4, 'colorTemperature'),
 (5, 'contact'),
 (6, 'level'),
 (7, 'lock'),
 (8, 'motion'),
 (9, 'on/off'),
 (10, 'ping'),
 (11, 'raw'),
 (12, 'read_attr_-_raw'),
 (13, 'status'),
 (14, 'switch'),
 (15, 'temperature'),
 (16, 'threeAxis'),
 (17, 'water')]

In [54]:
[ (i , classes[i]) for i in range( len(classes) ) ]

[(0, 'acceleration'),
 (1, 'activity'),
 (2, 'battery'),
 (3, 'button'),
 (4, 'colorTemperature'),
 (5, 'contact'),
 (6, 'level'),
 (7, 'lock'),
 (8, 'motion'),
 (9, 'ping'),
 (10, 'status'),
 (11, 'switch'),
 (12, 'temperature'),
 (13, 'threeAxis'),
 (14, 'water')]

In [42]:
def is_clean_event( inp, return_clean= True  ):
    if return_clean:
        return  'lock-locked' not in inp and 'lock-unlocked' not in inp and 'on/off-XXX' not in inp and 'raw-XXX' not in inp and 'read_attr_-_raw-XXX' not in inp
    else:
        return  'lock-locked' in inp or 'lock-unlocked'  in inp or 'on/off-XXX' in inp or  'raw-XXX' in inp  or 'read_attr_-_raw-XXX' in inp 
    
def is_clean_service( inp, return_clean= True  ):
    
    if return_clean:
        return  'on/off' not in inp and 'raw' not in inp and 'read_attr_-_raw' not in inp
    else:
        return  'on/off' in inp or  'raw' in inp  or 'read_attr_-_raw' in inp 
#     return  "contact-open" not in inp and 'contact-closed' not in inp

In [43]:

y_test[0][0]

array(['temperature'], dtype='<U11')

In [44]:
toKeep = [ i for i in range(len(y_train)) if is_clean_event( y_train[i]) ] if Mapper=='SE' else [ i for i in range(len(y_train)) if is_clean_service( y_train[i]) ]
x_train= [ x_train[i] for i in toKeep ]
y_train= [ y_train[i] for i in toKeep ]


In [45]:
for j in range(len(x_test)):
    toChange= [ i for i in range(len(y_test[j])) if is_clean_service( y_test[j][i], False) ]
    y_test[j] = [ (y_test[j][i] if i not in toChange else np.array( ['none'])) for i in range(len(y_test[j])) ]

In [46]:
# classes.remove('read_attr_-_raw-XXX')
# classes.remove('on/off-XXX')
# classes.remove('raw-XXX')
# classes.remove('lock-unlocked')
# classes.remove('lock-locked')


classes.remove('read_attr_-_raw')
classes.remove('on/off')
classes.remove('raw')


# classes.remove('lock')
# # classes.remove('lock')


# classes.remove('switch-on')

In [47]:
def pre_process_raw( x_data,y_data, dim_size = 128, zero_pad = False, normalize = False ,classes=None ):
#  y data 
    if classes is None:
        classes  = sorted(list(np.unique(  np.concatenate( y_data  ))))
    else :
        classes = sorted(classes)
    y_data_categorical = []  

    for x in y_data:
        temp = np.zeros( len(classes) )
        for y in x : 
            if y in classes:
                temp[ classes.index( y ) ] = 1
        y_data_categorical.append( temp )
    y_data_categorical = np.vstack(y_data_categorical)

#     x_data = np.array( x_data) / 1500.0
    
    x_data_temp = [] 
    
    if not zero_pad:
        for x in x_data:
            temp = [] #list(x)
            lst = list(x)
            while dim_size**2 - len(temp )   > len(lst):
                temp.extend(lst)

            while len(temp) < dim_size**2:
                temp.append( 0 )

            x_data_temp.append(np.array( temp).reshape(dim_size,dim_size))


        x_data_temp = np.array( x_data_temp )
        x_data_temp=x_data_temp.reshape(x_data_temp.shape+(1,))
    else :
        x_data_temp = sequence.pad_sequences(x_data, maxlen=dim_size)
    
    
    if normalize:
        x_data_temp = np.array( x_data_temp) / 1500.0
    else :
        x_data_temp = np.array(x_data_temp)
    
    
    return x_data_temp ,y_data_categorical , classes

# Model Evaluation Methods

In [48]:
y_lstm_prossed_test[2]

NameError: name 'y_lstm_prossed_test' is not defined

In [49]:
temp = np.sum( y_lstm_prossed_test.astype(int) , axis=0)
temp

NameError: name 'y_lstm_prossed_test' is not defined

In [1]:
def make_recall_shit( inp ):
    tp = inp[1][1]
    tn = inp[0][0]
    fp = inp[0][1] 
    fn = inp[1][0]
    
    acc = (tp+tn)*1.0 / ( tp+tn+fp+fn)*1.0
    recall = tp*1.0/ ( tp+fn ) *1.0
    prec = tp*1.0 / ( tp+fp )*1.0
    
#     F= 2.0*( prec* recall )/ (prec+recall)
    F= 2.0*( tp)/ (2*tp + fp + fn)
    
    return acc, recall, prec, F

def acc_match( true, pred ):
    """
    returns exact mathc accuracy
    """
 
    return (len( [ x  for x  in  [np.sum(np.abs( true[i]- pred[i] )) for i in range(len(true))] if x  == 0]))*1.0 / len(true)


def acc_match_wierd( true, pred ):
    """
    returns exact mathc accuracy
    """
    level = 6 
    switch = 11
    threeAxis=13
    accel = 0 
    status=10
    contact=5
    
    counter  = 0 
    for i in range( len (true) ):
        if np.sum(np.abs( true[i]- pred[i] ))==0 :
            counter+=1
        else : 
            t_rec = np.array(list( pred[i]))
            
            if true[i][level]==1 and true[i][switch]==1 and pred[i][level]==1 :
                pred[i][switch]=1
            
            if true[i][threeAxis]==1 and true[i][accel]==1 and pred[i][threeAxis]==1:
                pred[i][accel] =1
            
            if true[i][status]==1 and true[i][contact]==1 and pred[i][status]==1:
                pred[i][contact]=1
#             print(t_rec , true[i])    
            if np.sum(np.abs( true[i]- pred[i] ))==0 :
                counter+=1   
            
             
            
    
    return counter*1.0 / len(true)


def print_info(y_test, pred , classes , confidance=0.5 ):
    
    counts = np.sum( y_test.astype(int) , axis=0)
    
    pred[pred>=confidance] = 1
    pred[pred<confidance] = 0
    
    acc_wierd  =acc_match_wierd(y_test, pred)
    
    conf= multilabel_confusion_matrix( y_test , pred.astype(int), labels= range(len(classes)))
    accs = [make_recall_shit(x) for x in conf]
    print( "%30s  %8s   %8s  %8s  %8s %8s %16s"  %( "Class","Accuracy", "Recall","Precision","F Score" , "Count", "TP/TN/FP/FN"))
    print( "------------------------------------------------------------------------" )
    for index in range(len(classes)):
        tp = conf[index][1][1]
        tn = conf[index][0][0]
        fp = conf[index][0][1] 
        fn = conf[index][1][0]
        print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %d/%d/%d/%d"  %
             (classes[index],
              accs[index][0],
              accs[index][1],
              accs[index][2],
              accs[index][3],
              counts[index],
                  tp ,
    tn ,
    fp ,
    fn 
             )

           
             )
    
    n_zeros_true = len([ x  for x  in  [np.sum(np.abs( y_test[i] )) for i in range(len(y_test))] if x  == 0]  )
    n_zeros_pred = len([ x  for x  in  [np.sum(np.abs( pred[i] )) for i in range(len(pred))] if x  == 0]  )
    
    print ("------------------------------------------------------------------------")
    print ( "Exact Match ACC : %.5f " % acc_match( y_test, pred )  )
    print ( "Wierd Exact Match ACC : %.5f" % acc_wierd)
    print ( "Total Records : %d " % len(y_test)  )
    print ( "Total ZXeros in True : %d (%.3f)%%" % (n_zeros_true ,  n_zeros_true * 1.0/ len(y_test)  ))
    print ( "Total ZXeros in Test : %d (%.3f)%%" % (n_zeros_pred ,  n_zeros_pred * 1.0/ len(y_test)  ) )
    print ('=============================================================================')
    

In [74]:
def make_readable_results ( inp , classes , conffidance=True):
    ret = [] 
    inp =inp.astype(int)
    for xx in range(len(inp)) :
        u = inp[xx]
        temp = []
        for j in range(len(u)) : 
            if u[j] >0:
                temp.append(classes[j])
        ret.append(temp)
    return ret


def makeReadable( model , data, gt, path , classes, x, confidance=0.7):
    pred_temp = model.predict(data)
    print_info(gt, pred_temp, classes , confidance=confidance)
    print( len(classes ), len( pred_temp[0] ) )
    xcc= make_readable_results(pred_temp , classes)
    y_gt = make_readable_results( gt, classes )
    temp_dic = {} 
    for pick in range(len(xcc)): 
        temp_dic[ pick +1 ] =  { 'seq': str(data[pick]),
                               'pred': xcc[pick],
                                'true':y_gt[pick]
                               }   

    with open(path , 'w') as f:
        json.dump(temp_dic , f, indent=4)


# def makeReadable( model , data, gt, path , classes, confidance=0.7, x):
#     pred_temp = model.predict( data)
#     print_info(gt, pred_temp, classes , confidance=confidance)
#     xcc= make_readable_results( pred_temp , classes )
#     temp_dic = {} 
#     for pick in range(len(xcc)): 
#         temp_dic[ pick +1 ] = xcc[pick]  

#     with open(path , 'w') as f:
#         json.dump(temp_dic , f, indent=4)

# Create the Model 

In [ ]:
import numpy as np
import os
import tensorflow as tf
from keras.models import Model

from keras.layers import Input, merge, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, Dropout, Conv2DTranspose, UpSampling2D, Lambda
from keras.layers import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization as bn
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import RMSprop
from keras import regularizers
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers.merge import add
from keras.models import load_model
import numpy as np
from keras.regularizers import l2

def unet(dim_size , classcount):
    l2_lambda = 0.0002
    DropP = 0.3
    kernel_size=3

    inputs = Input((dim_size,dim_size,1))
    
    
    conv1 = Conv2D( 32, (kernel_size, kernel_size), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(inputs)
    conv1 = bn()(conv1)
    conv1 = Conv2D(32, (kernel_size, kernel_size), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv1)
    conv1 = bn()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(DropP)(pool1)





    conv2 = Conv2D(64, (kernel_size, kernel_size), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(pool1)
    conv2 = bn()(conv2)
    conv2 = Conv2D(64, (kernel_size, kernel_size), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv2)
    conv2 = bn()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(DropP)(pool2)



    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(pool2)
    conv3 = bn()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv3)
    conv3 = bn()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(DropP)(pool3)



    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(pool3)
    conv4 = bn()(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv4)
    conv4 = bn()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = Dropout(DropP)(pool4)



    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(pool4)
    conv5 = bn()(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv5)
    conv5 = bn()(conv5)
    
    up6 = concatenate([Conv2DTranspose(256,(2, 2), strides=(2, 2), padding='same')(conv5), conv4],name='up6', axis=3)
    up6 = Dropout(DropP)(up6)
    conv6 = Conv2D(256,(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(up6)
    conv6 = bn()(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv6)

    conv6 = bn()(conv6)
    up7 = concatenate([Conv2DTranspose(128,(2, 2), strides=(2, 2), padding='same')(conv6), conv3],name='up7', axis=3)
    up7 = Dropout(DropP)(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(up7)
    conv7 = bn()(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv7)
    conv7 = bn()(conv7)

    up8 = concatenate([Conv2DTranspose(64,(2, 2), strides=(2, 2), padding='same')(conv7), conv2],name='up8', axis=3)
    up8 = Dropout(DropP)(up8)
    conv8 = Conv2D(64, (kernel_size, kernel_size), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(up8)
    conv8 = bn()(conv8)
    conv8 = Conv2D(64, (kernel_size, kernel_size), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv8)
    conv8 = bn()(conv8)

    up9 = concatenate([Conv2DTranspose(32,(2, 2), strides=(2, 2), padding='same')(conv8), conv1],name='up9',axis=3)
    up9 = Dropout(DropP)(up9)
    conv9 = Conv2D(32, (kernel_size, kernel_size), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(up9)
    conv9 = bn()(conv9)
    conv9 = Conv2D(32, (kernel_size, kernel_size), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv9)
    conv9 = bn()(conv9) 
    
    
    
    flatten1 = Flatten()(conv9)
    conv7 = Dense( 256, activation='relu' )(flatten1)
    conv8 = Dense( 128, activation='relu' )(conv7)
    
    
    output = Dense(classcount,  activation='sigmoid',name='output')(conv8)
    
    model = Model(inputs=[inputs], outputs=[output])
    model.compile(optimizer=Adam(lr=1e-5), loss=weighted_categorical_crossentropy(weights=weights), metrics=['accuracy'])
    return model

In [ ]:
dim_size =32
x_train_processed,y_train_processed, _ = pre_process_raw( x_train, y_train , dim_size, classes=classes)
x_test_processed,y_test_processed, _ = pre_process_raw( x_test[0], y_test[0], dim_size, classes=classes)


In [ ]:
model2 = unet(dim_size,len( y_train_processed[0]))
# model2.summary()


In [ ]:
model2.save("UNET_Sigmoid_200Epoch_trainedWithoutSwithces")

 a
 

In [ ]:
report2=  model2.fit( x=x_train_processed , y=y_train_processed , batch_size=1800, epochs=100 )

In [ ]:
model2.save( "UNET_LargeData_F%s_E%d_B%d_M%s_%r" %
            (
            FoldID,
                Epoch_count,
                Batch_size,
                Mapper,
                IgnoreEmpty
            ) 
           )

In [ ]:
plt.plot(report2.history['loss'])

In [ ]:
pred= model2.predict( x_test_processed )


In [ ]:
len([x for x in pred if np.sum(x) < 1 ])

In [ ]:
len(pred[0]),len( y_test_processed[0]),len(classes)

In [ ]:
print_info(y_test_processed, pred, classes,confidance=0.9)

In [ ]:
pred[0]

In [ ]:
print_info(y_test_processed, pred, classes,confidance=0.6)

In [ ]:
print_info(y_test_processed, pred, classes,confidance=0.6)

# Calcualte per class accuracy

In [ ]:
# try:
#     print(X_test is None)
#     np.save( "x_test.npy", X_test )
#     np.save("y_test.npy", y_test)
#     np.save("y_pred.npy", pred )
#     np.save("classes.npy", classes)
# except:
#     X_test = np.load("x_test.npy")
#     y_test = np.load("y_test.npy")
#     pred   = np.load("y_pred.npy")
#     classes = np.load("classes.npy")

# Random Forest baseline calc

In [52]:
dim_size= 50
x_random_forest_train,y_random_forest_train, _ = pre_process_raw( x_train, y_train , dim_size, zero_pad=True, normalize=False, classes=classes)
# x_random_forest_test,y_random_forest_test, _ = pre_process_raw( x_test[0], y_test[0] , dim_size, zero_pad=True, normalize=False, classes=classes)
rf_tests  = [ pre_process_raw( x_test[i], y_test[i] , dim_size, zero_pad=True, normalize=False, classes=classes) for i in range(len(x_test)) ] 
# x,y, classes = pre_process_raw( x_data, y_data , dim_size, zero_pad=True, normalize=True)

In [53]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=960, max_depth=9050,
                             random_state=0 )
t_hist = clf.fit(x_random_forest_train, y_random_forest_train)




# print(clf.feature_importances_)

# print(clf.predict([[0, 0, 0, 0]]))
# from sklearn import metrics
# scores = cross_val_score(clf, x_random_forest_train, y_random_forest_train, cv=10, scoring='accuracy')

In [59]:
for i in range(len(rf_tests)) :
    print( "==================HOME Case : %s =============" % test_names[ i] )
    rf_pred= clf.predict( rf_tests[i][0])
    print_info( rf_tests[i][1], rf_pred, classes)

==================HOME Case : fixed_home_muhammed.json =============


/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


                         Class  Accuracy     Recall  Precision   F Score    Count
------------------------------------------------------------------------
                  acceleration     0.977      0.000     0.000     0.000         5 
                      activity     0.998      0.074     0.308     0.119        54 
                       battery     0.998        nan     0.000     0.000         0 
                        button     0.990      0.000     0.000     0.000         5 
              colorTemperature     0.998        nan     0.000     0.000         0 
                       contact     0.943      0.482     0.017     0.033        56 
                         level     0.957      0.765     0.011     0.021        17 
                          lock     0.953      0.222     0.002     0.003         9 
                        motion     0.906      0.504     0.201     0.287      1045 
                          ping     0.754        nan     0.000     0.000         0 
               

/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


                         Class  Accuracy     Recall  Precision   F Score    Count
------------------------------------------------------------------------
                  acceleration     0.978      0.036     0.006     0.010        55 
                      activity     0.999      0.059     0.500     0.105        17 
                       battery     0.998      0.000     0.000     0.000         7 
                        button     0.994      0.000     0.000     0.000         9 
              colorTemperature     1.000      0.800     0.400     0.533         5 
                       contact     0.940      0.517     0.070     0.124       145 
                         level     0.983      0.938     0.049     0.093        16 
                          lock     0.948      0.559     0.021     0.040        34 
                        motion     0.921      0.528     0.126     0.203       335 
                          ping     0.731        nan     0.000     0.000         0 
               

...

/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


                         Class  Accuracy     Recall  Precision   F Score    Count
------------------------------------------------------------------------
                  acceleration     0.966      0.000     0.000     0.000         6 
                      activity     0.998      0.259     0.667     0.373        54 
                       battery     0.997        nan     0.000     0.000         0 
                        button     0.991      0.000     0.000     0.000         6 
              colorTemperature     0.995        nan     0.000     0.000         0 
                       contact     0.947      0.474     0.022     0.042        57 
                         level     0.963      0.650     0.015     0.030        20 
                          lock     0.958      0.200     0.002     0.004        10 
                        motion     0.908      0.365     0.199     0.258      1011 
                          ping     0.729        nan     0.000     0.000         0 
               

/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


                         Class  Accuracy     Recall  Precision   F Score    Count
------------------------------------------------------------------------
                  acceleration     0.960      0.089     0.009     0.017        56 
                      activity     0.999      0.235     0.667     0.348        17 
                       battery     0.997      0.143     0.025     0.043         7 
                        button     0.993      0.000     0.000     0.000        11 
              colorTemperature     0.999      0.800     0.286     0.421         5 
                       contact     0.951      0.542     0.105     0.177       142 
                         level     0.983      0.750     0.059     0.109        20 
                          lock     0.949      0.588     0.027     0.051        34 
                        motion     0.927      0.398     0.130     0.196       324 
                          ping     0.699        nan     0.000     0.000         0 
               

/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


In [ ]:
makeReadable( data=rf_tests[0][0], model=clf, classes=classes, confidance=0.7,gt=rf_tests[0][1], path='sk_home_out.json' )

In [ ]:
print_info(y_random_forest_test, rf_pred, classes)

In [ ]:
# print_info(y_random_forest_test, rf_pred, classes)

In [ ]:
# print ( "mean : %f \nstd: %f\nmax:%f" %( scores.mean(), scores.std(), scores.max()) )

# LSTM 

In [ ]:
cnts  = np.unique(np.array([ len(x) for x  in x_train ]), return_counts=True)
# np.sort( cnts[1] )
cnts

In [76]:
# create the model
dim_size =15
x_lstm_prossed_train,y_lstm_prossed_train, _ = pre_process_raw( x_train, y_train , dim_size, zero_pad=True, normalize=False,classes=classes)
# x_lstm_prossed_test,y_lstm_prossed_test, _ = pre_process_raw( x_test, y_test_2 , dim_size, zero_pad=True, normalize=False,classes=classes)
lstm_tests  = [ pre_process_raw( x_test[i], y_test[i] , dim_size, zero_pad=True, normalize=False, classes=classes) for i in range(len(x_test)) ] 


In [77]:
len(lstm_tests[0][0])

27720

In [78]:
# x_lstm_prossed_test2 = np.expand_dims(x_lstm_prossed_test,axis=1)
# x_lstm_prossed_train2 =np.expand_dims(x_lstm_prossed_train,axis=1)

for tt  in range( len(lstm_tests) ):
    lstm_tests[tt]= (lstm_tests[tt][0].reshape(len(lstm_tests[tt][0]),dim_size,1) , lstm_tests[tt][1],None )
# x_lstm_prossed_test2 = x_lstm_prossed_test.reshape(len(x_lstm_prossed_test),dim_size,1)
x_lstm_prossed_train2 =x_lstm_prossed_train.reshape(len(x_lstm_prossed_train),dim_size,1)

# y_lstm_prossed_test2 = y_lstm_prossed_test.reshape(len(y_lstm_prossed_test),len(classes),1)
# y_lstm_prossed_train2 =y_lstm_prossed_train.reshape(len(y_lstm_prossed_train),len(classes),1)

In [79]:
weights =np.array([1.0/(50.0/50358.0),
1.0/(52.0/50358.0),
1.0/(3.0/50358.0),
1.0/(5.0/50358.0),
1.0/(4.0/50358.0),
1.0/(11.0/50358.0),
1.0/(58.0/50358.0),
1.0/(210.0/50358.0),
1.0/(376.0/50358.0),
1.0/(118.0/50358.0),
1.0/(121.0/50358.0),
1.0/(694.0/50358.0),
1.0/(230.0/50358.0),
1.0/(221.0/50358.0),
1.0/(105.0/50358.0),
1.0/(98.0/50358.0),
1.0/(16560.0/50358.0),
1.0/(30490.0/50358.0),
1.0/(94.0/50358.0),
1.0/(96.0/50358.0),
1.0/(452.0/50358.0),
1.0/(68.0/50358.0),
1.0/(50.0/50358.0),
1.0/(45.0/50358.0),])
weights

array([1.00716000e+03, 9.68423077e+02, 1.67860000e+04, 1.00716000e+04,
       1.25895000e+04, 4.57800000e+03, 8.68241379e+02, 2.39800000e+02,
       1.33930851e+02, 4.26762712e+02, 4.16181818e+02, 7.25619597e+01,
       2.18947826e+02, 2.27864253e+02, 4.79600000e+02, 5.13857143e+02,
       3.04094203e+00, 1.65162348e+00, 5.35723404e+02, 5.24562500e+02,
       1.11411504e+02, 7.40558824e+02, 1.00716000e+03, 1.11906667e+03])

In [80]:

# weights =np.array([1.0/(50.0/50358.0),
# 1.0/(52.0/50358.0),
# 1.0/(3.0/50358.0),
# 1.0/(5.0/50358.0),
# 1.0/(4.0/50358.0),
# 1.0/(11.0/50358.0),
# 1.0/(58.0/50358.0),
# 1.0/(210.0/50358.0),
# 1.0/(376.0/50358.0),
# 1.0/(118.0/50358.0),
# 1.0/(121.0/50358.0),
# 1.0/(694.0/50358.0),
# 1.0/(230.0/50358.0),
# 1.0/(221.0/50358.0),
# 1.0/(105.0/50358.0),
# 1.0/(98.0/50358.0),
# 1.0/(16560.0/50358.0),
# 1.0/(30490.0/50358.0),
# 1.0/(94.0/50358.0),
# 1.0/(96.0/50358.0),
# 1.0/(452.0/50358.0),
# 1.0/(68.0/50358.0),
# 1.0/(50.0/50358.0),
# 1.0/(45.0/50358.0),])
weights =np.array([
1.0, # 50.0
1.0, # 52.0
1.5, # 3.0
1.5, # 5.0
1.5, # 4.0
1.5, # 11.0
1.0, # 58.0
0.80, # 210.0
0.80, # 376.0
0.80, # 118.0
0.80, # 121.0
0.70, # 694.0
0.80, # 230.0
0.80, # 221.0
0.90, # 105.0
0.90, # 98.0
0.40, # 16562.0
0.40, # 30490.0
0.90, # 94.0
0.90, # 96.0
0.80, # 452.0
0.90, # 68.0
0.90, # 50.0
0.90  # 45.0
])

weights = weights*15

In [90]:
y_train[0]

array(['level', 'switch'], dtype='<U6')

In [93]:
# len(classes)
nn = [len(c) for c  in y_train]
np.max(nn)
# np.max((len(y_train[i]) for i in range(len( y_train ))))


4

In [82]:
## Bidirectional LSTM :D 
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Flatten


import numpy as np
import os
import tensorflow as tf
from keras.models import Model

from keras.layers import Input, merge, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, Dropout, Conv2DTranspose, UpSampling2D, Lambda
from keras.layers import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization as bn
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import RMSprop
from keras import regularizers
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers.merge import add
from keras.models import load_model
import numpy as np
from keras.regularizers import l2
from keras.layers import Conv1D
from keras.layers.merge import add
from keras.layers import Dense,Conv1D,Dropout,Activation,BatchNormalization,MaxPooling1D,Flatten,Masking,TimeDistributed
from keras.layers.recurrent import LSTM,GRU,SimpleRNN
from keras.models import Input,Sequential,Model
from keras.layers.merge import add
from keras.optimizers import Adam
from keras.losses import MSE,MSLE

model2 = Sequential()


def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

weights =np.array([1.0/(50.0/50358.0),
1.0/(52.0/50358.0),
1.0/(3.0/50358.0),
1.0/(5.0/50358.0),
1.0/(4.0/50358.0),
1.0/(11.0/50358.0),
1.0/(58.0/50358.0),
1.0/(210.0/50358.0),
1.0/(376.0/50358.0),
1.0/(118.0/50358.0),
1.0/(121.0/50358.0),
1.0/(694.0/50358.0),
1.0/(230.0/50358.0),
1.0/(221.0/50358.0),
1.0/(105.0/50358.0),
1.0/(98.0/50358.0),
1.0/(16560.0/50358.0),
1.0/(30490.0/50358.0),
1.0/(94.0/50358.0),
1.0/(96.0/50358.0),
1.0/(452.0/50358.0),
1.0/(68.0/50358.0),
1.0/(50.0/50358.0),
1.0/(45.0/50358.0),])
# weights = weights/ np.max(weights)

# model2.add(LSTM(50, input_shape=(dim_size, 1), return_sequences=True))
# model2.add(TimeDistributed(Dense(1, activation='relu')))
# model2.add(Dropout(0.2))
# model2.add(Flatten())
# model2.add(Dense(127, activation='relu'))
# model2.add(Dense(len(classes), activation='sigmoid'))
# # model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])


# # model2.add(Embedding(word_count, dim_size, input_length=dim_size))
# # model2.add(Dropout(0.1))

inputs  = Input( ( dim_size,1 ) )

# bi_d_1 = Bidirectional( LSTM(200 ,  recurrent_dropout=0.12, return_sequences=True),input_shape=[dim_size,1],merge_mode='concat') (inputs)
lstm_1 =  LSTM(60 ,  recurrent_dropout=0.04, return_sequences=True)(inputs)
# lstm_2 = LSTM(30 ,  recurrent_dropout=0.14, return_sequences=True)(lstm_1)

bi_d_1 =Dense(128, activation='relu')  (lstm_1)
lstm_1 =  Dense(128, activation='relu')(bi_d_1)
lstm_2 = Dense(128, activation='relu')(lstm_1)



# td_1    = TimeDistributed(Dense(256, activation='relu'))(lstm_2)
# dout_1  = Dropout(0.1)(td_1)
dout_1  = Dropout(0.1)(lstm_2)
flt_1   = Flatten()(dout_1)
dense_1 = Dense(128, activation='relu')(flt_1)
dout_2  = Dropout(0.2)(dense_1)


out = Conv1D(128,3,padding='same')(inputs)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = Dropout(0.2)(out)
out = Conv1D(128,3,padding='same')(out)
# out = Flatten()(out)
# out = MaxPooling1D(2,padding='same', name ='pooling')(out)


out = Conv1D(128,3,padding='same')(out)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = Dropout(0.2)(out)
out = Conv1D(128,3,padding='same')(out)
# out = Flatten()(out)
# out = MaxPooling1D(2,padding='same', name ='pooling2')(out)


out = Conv1D(128,3,padding='same')(out)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = Dropout(0.2)(out)
out = Conv1D(128,3,padding='same')(out)
# out = Flatten()(out)
# out = MaxPooling1D(2,padding='same', name ='pooling')(out)



# fl_out_1 = Flatten()(dout_2)

fl_out_cnn = Flatten()(out)

# out_new = concatenate( [fl_out_1, fl_out_cnn] , name='mergerguy')
out_new = concatenate( [dout_2, fl_out_cnn] , name='mergerguy')

dens_out_1 = Dense( 128, activation='relu' )(out_new)
dens_out_2 = Dense( 128, activation='relu' )(dens_out_1)
dens_out_3 = Dense( 128, activation='relu' )(dens_out_2)

# fl2  = Flatten()(out_new)

out_put_final = Dense(len(classes), activation='sigmoid', name='output_finale')(dens_out_3)




model2 = Model(inputs=[inputs], outputs=[out_put_final])




# model2.add(Bidirectional( LSTM(200 ,  recurrent_dropout=0.12, return_sequences=True),input_shape=[dim_size,1],merge_mode='concat'))
# model2.add( LSTM(60 ,  recurrent_dropout=0.04, return_sequences=True))
# model2.add( LSTM(30 ,  recurrent_dropout=0.14, return_sequences=True))
# # model2.add(Bidirectional( LSTM(100 ,  recurrent_dropout=0.04, return_sequences=True),merge_mode='concat'))
# model2.add(TimeDistributed(Dense(256, activation='relu')))
# model2.add(Dropout(0.1))
# model2.add(Flatten())
# model2.add(Dense(128, activation='relu'))
# model2.add(Dropout(0.2))
# model2.add(Dense(len(classes), activation='sigmoid'))

# model2.compile(loss=weighted_categorical_crossentropy(weights=weights), optimizer='adam', metrics=['accuracy'])

model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint = ModelCheckpoint('IoTDownNet', monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

print(model2.summary())
hist2 = model2.fit(x_lstm_prossed_train2, y_lstm_prossed_train, epochs=100, batch_size=1500, shuffle=True, callbacks=callbacks_list)





____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 15, 1)         0                                            
____________________________________________________________________________________________________
conv1d_7 (Conv1D)                (None, 15, 128)       512         input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_4 (BatchNorm (None, 15, 128)       512         conv1d_7[0][0]                   
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 15, 128)       0           batch_normalization_4[0][0]      
___________________________________________________________________________________________

In [83]:
hist2 = model2.fit(x_lstm_prossed_train2, y_lstm_prossed_train, epochs=300, batch_size=8500, shuffle=True, callbacks=callbacks_list)

Epoch 1/300
7816/7816 [==============================] - 0s - loss: 0.9349 - acc: 0.6444
Epoch 2/300
7816/7816 [==============================] - 0s - loss: 0.9318 - acc: 0.6626
Epoch 3/300
7816/7816 [==============================] - 0s - loss: 0.9334 - acc: 0.6717
Epoch 4/300
7816/7816 [==============================] - 0s - loss: 0.9303 - acc: 0.6768
Epoch 5/300
7816/7816 [==============================] - 0s - loss: 0.9318 - acc: 0.6849
Epoch 6/300
7816/7816 [==============================] - 0s - loss: 0.9303 - acc: 0.6803
Epoch 7/300
7816/7816 [==============================] - 0s - loss: 0.9315 - acc: 0.6762
Epoch 8/300
7816/7816 [==============================] - 0s - loss: 0.9305 - acc: 0.6745
Epoch 9/300
7816/7816 [==============================] - 0s - loss: 0.9291 - acc: 0.6807
Epoch 10/300
7816/7816 [==============================] - 0s - loss: 0.9283 - acc: 0.6836
Epoch 11/300
7816/7816 [==============================] - 0s - loss: 0.9253 - acc: 0.6871
Epoch 12/300
7816/7

7816/7816 [==============================] - 0s - loss: 0.9118 - acc: 0.6436
Epoch 93/300
7816/7816 [==============================] - 0s - loss: 0.9121 - acc: 0.6424
Epoch 94/300
7816/7816 [==============================] - 0s - loss: 0.9107 - acc: 0.6478
Epoch 95/300
7816/7816 [==============================] - 0s - loss: 0.9107 - acc: 0.6491
Epoch 96/300
7816/7816 [==============================] - 0s - loss: 0.9138 - acc: 0.6448
Epoch 97/300
7816/7816 [==============================] - 0s - loss: 0.9094 - acc: 0.6461
Epoch 98/300
7816/7816 [==============================] - 0s - loss: 0.9102 - acc: 0.6439
Epoch 99/300
7816/7816 [==============================] - 0s - loss: 0.9075 - acc: 0.6433
Epoch 100/300
7816/7816 [==============================] - 0s - loss: 0.9099 - acc: 0.6455
Epoch 101/300
7816/7816 [==============================] - 0s - loss: 0.9090 - acc: 0.6433
Epoch 102/300
7816/7816 [==============================] - 0s - loss: 0.9083 - acc: 0.6480
Epoch 103/300
7816/7

7816/7816 [==============================] - 0s - loss: 0.8963 - acc: 0.6755
Epoch 273/300
7816/7816 [==============================] - 0s - loss: 0.8891 - acc: 0.6772
Epoch 274/300
7816/7816 [==============================] - 0s - loss: 0.8890 - acc: 0.6746
Epoch 275/300
7816/7816 [==============================] - 0s - loss: 0.8931 - acc: 0.6743
Epoch 276/300
7816/7816 [==============================] - 0s - loss: 0.8881 - acc: 0.6741
Epoch 277/300
7816/7816 [==============================] - 0s - loss: 0.8952 - acc: 0.6721
Epoch 278/300
7816/7816 [==============================] - 0s - loss: 0.8884 - acc: 0.6720
Epoch 279/300
7816/7816 [==============================] - 0s - loss: 0.8885 - acc: 0.6717
Epoch 280/300
7816/7816 [==============================] - 0s - loss: 0.8869 - acc: 0.6723
Epoch 281/300
7816/7816 [==============================] - 0s - loss: 0.8825 - acc: 0.6739
Epoch 282/300
7816/7816 [==============================] - 0s - loss: 0.8898 - acc: 0.6730
Epoch 283/300

In [ ]:

# word_count = len(np.max( x_lstm_prossed_train))+1 # len(np.unique( x ))

# create the model
embedding_vecor_length = 102
model2 = Sequential()
# model2.add(Embedding(word_count, dim_size, input_length=dim_size))
# model2.add(Dropout(0.1))
model2.add(LSTM(100 , dropout=0.2, recurrent_dropout=0.2, input_shape=[1,dim_size]))
model2.add(Dropout(0.1))
model2.add(Dense(127, activation='relu'))
model2.add(Dense(len(classes), activation='sigmoid'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())
hist2 = model2.fit(x_lstm_prossed_train2, y_lstm_prossed_train, epochs=300, batch_size=1500)




# word_count = len(np.unique( x_lstm_prossed_train))+1 # len(np.unique( x ))

# # create the model
# embedding_vecor_length = 102
# model2 = Sequential()
# model2.add(Embedding(word_count, dim_size, input_length=dim_size))
# model2.add(Dropout(0.1))
# model2.add(LSTM(100 , dropout=0.2, recurrent_dropout=0.2))

# model2.add(Dense(127, activation='relu'))
# model2.add(Dense(len(classes), activation='sigmoid'))
# model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model2.summary())
# hist2 = model2.fit(x_lstm_prossed_train, y_lstm_prossed_train, epochs=300, batch_size=1500)


# word_count = np.max( x_lstm_prossed_train)+1 # len(np.unique( x ))

# # create the model
# embedding_vecor_length = 102
# model2 = Sequential()
# model2.add(Embedding(word_count, embedding_vecor_length, input_length=dim_size))
# model2.add(Dropout(0.1))
# model2.add(LSTM(100))

# model2.add(Dense(127, activation='relu'))
# model2.add(Dense(len(classes), activation='sigmoid'))
# model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model2.summary())
# hist2 = model2.fit(x_lstm_prossed_train, y_lstm_prossed_train, epochs=Epoch_count, batch_size=Batch_size)

In [ ]:
model2.compile(loss=weighted_categorical_crossentropy(weights=weights), optimizer=keras.optimizers.Adam(lr=8e-5  ), metrics=['accuracy'])

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=8e-5  ), metrics=['accuracy'])

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model2.compile(loss=weighted_categorical_crossentropy(weights=weights), optimizer='adam', metrics=['accuracy'])

In [ ]:
hist2 = model2.fit(x_lstm_prossed_train2, y_lstm_prossed_train, epochs=10000, batch_size=3500, shuffle=True)

In [ ]:
model2.save( "LSTM-sigmoid-withRemovedClasses" )

In [ ]:
plt.plot(hist2.history['loss'], c='red')
plt.plot(hist2.history['acc'], c='blue')

In [ ]:
# from keras.models import load_model
# model2=load_model( "LSTM_withSigmoid_LargeData_F1_E100_B500_MSE_False"  
#            )

In [86]:
for i in range(len(lstm_tests)) :
    print( "==================HOME Case : %s =============" % test_names[i])
    makeReadable( classes=classes, confidance=0.7,data=lstm_tests[i][0],gt=lstm_tests[i][1],model=model2,path=test_names[i],x=lstm_tests[i][0])
    
#     lstm_pred= model2.predict( lstm_tests[i][0])
    

==================HOME Case : fixed_home_muhammed.json =============


/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


                         Class  Accuracy     Recall  Precision   F Score    Count
------------------------------------------------------------------------
                  acceleration     0.994      0.200     0.006     0.013         5 
                      activity     0.992      0.019     0.006     0.009        54 
                       battery     1.000        nan     0.000     0.000         0 
                        button     0.990      0.000     0.000     0.000         5 
              colorTemperature     0.994        nan     0.000     0.000         0 
                       contact     0.985      0.500     0.068     0.119        56 
                         level     0.926      0.647     0.005     0.011        17 
                          lock     0.921      0.000     0.000     0.000         9 
                        motion     0.919      0.211     0.135     0.164      1045 
                          ping     0.950        nan     0.000     0.000         0 
               

/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


                         Class  Accuracy     Recall  Precision   F Score    Count
------------------------------------------------------------------------
                  acceleration     0.992      0.075     0.214     0.111        40 
                      activity     1.000        nan       nan       nan         0 
                       battery     1.000      0.000       nan     0.000         2 
                        button     0.997      0.238     0.556     0.333        21 
              colorTemperature     0.999      0.000     0.000     0.000         1 
                       contact     0.984      0.489     0.730     0.586       133 
                         level     0.956      0.462     0.048     0.087        26 
                          lock     0.947        nan     0.000     0.000         0 
                        motion     0.947      0.231     0.128     0.165       130 
                          ping     0.956        nan     0.000     0.000         0 
               

------------------------------------------------------------------------
Exact Match ACC : 0.38004 
Wierd Exact Match ACC : 0.38004
Total Records : 22974 
Total ZXeros in True : 20476 (0.891)%
Total ZXeros in Test : 8727 (0.380)%
15 15
==================HOME Case : rbf_home_os.json =============
                         Class  Accuracy     Recall  Precision   F Score    Count
------------------------------------------------------------------------
                  acceleration     0.994      0.125     0.175     0.146        56 
                      activity     0.999      0.000     0.000     0.000        17 
                       battery     0.999      0.000     0.000     0.000         7 
                        button     0.999      0.273     0.250     0.261        11 
              colorTemperature     0.999      0.800     0.222     0.348         5 
                       contact     0.974      0.570     0.206     0.302       142 
                         level     0.961      0.50

In [2]:
# for i in [0] :
for i in range(len(lstm_tests)) :
    print( "==================HOME Case : %s =============" % test_names[i])
    lstm_pred= model2.predict( lstm_tests[i][0])
    
    print_info( lstm_tests[i][1], lstm_pred, classes , confidance=0.7)
    



NameError: name 'lstm_tests' is not defined

In [ ]:
lstm_pred =model2.predict( x_lstm_prossed_test2)


In [ ]:
print_info(y_lstm_prossed_test, lstm_pred, classes, confidance=0.60)

In [ ]:
lstm_pred =model2.predict( x_lstm_prossed_test)
print_info(y_lstm_prossed_test, lstm_pred, classes, confidance=0.6)

In [ ]:
lstm_pred = model2.predict( x_lstm_prossed_test)
print_info(y_lstm_prossed_test, lstm_pred, classes, confidance=0.6)

In [ ]:
print_info(y_lstm_prossed_train, y_lstm_prossed_train, classes, confidance=0.60)

In [ ]:
[x for x  in lstm_pred  if  np.sum(x) > 0 ]

In [ ]:
model2.save( "LSTM_withSigmoid_LargeData_F%s_E%d_B%d_M%s_%r" %
            (
            FoldID,
                Epoch_count,
                Batch_size,
                Mapper,
                IgnoreEmpty
            ) 
           )

In [ ]:
lstm_pred = model2.predict( x_lstm_prossed_test )

In [ ]:
print_info(y_lstm_prossed_test, lstm_pred, classes)

In [ ]:
# x_train_muhammed,y_train_muhammed, classes = pre_process_raw( x_train, y_train , dim_size, zero_pad=True, normalize=False)
# x_test_muhammed,y_test_muhammed, classes = pre_process_raw( x_test, y_test , dim_size, zero_pad=True, normalize=False)

In [ ]:
lstm_pred = model2.predict( x_lstm_prossed_test )
print_info(y_lstm_prossed_test, lstm_pred, classes)

In [ ]:
lstm_pred = model2.predict( x_lstm_prossed_test )
print_info(y_lstm_prossed_test, lstm_pred, classes)

In [ ]:
pred[:10]

In [ ]:
print_info(y_lstm_prossed_test, pred, classes)

In [ ]:
pred[0]

In [ ]:
len( y_lstm_prossed_train[0] )

In [ ]:
dim_size =160
x_lstm_prossed_train,y_lstm_prossed_train, _ = pre_process_raw( x_train, y_train , dim_size, zero_pad=True, normalize=False,classes=classes)
x_lstm_prossed_test,y_lstm_prossed_test, _ = pre_process_raw( x_test, y_test , dim_size, zero_pad=True, normalize=False,classes=classes)

In [ ]:
len([x  for x  in y_lstm_prossed_test if x[21]==1 or x[20]==1]), len(y_lstm_prossed_test  ) , len([x  for x  in y_lstm_prossed_test if x[21]==1 or x[20]==1])/len(y_lstm_prossed_test  ) *1.0

In [ ]:
[ x for x  in  pred if np.sum(x) > 1]

In [ ]:
len(y_lstm_prossed_test)

In [ ]:
def do_for_raun( pred   ):
    pp = pred
    pp[pp>=0.5] = 1
    pp[pp<0.5] = 0
    return pp

In [ ]:
len([x for x in pred if np.sum( do_for_raun(x) )==0 ])

In [ ]:
len([x for x in pred if  do_for_raun(x)[20] ==1 or do_for_raun(x)[21] ==1 ])

In [ ]:

# np.save("../files/muhammed/x_train.json" , x_train_muhammed)
# np.save("../files/muhammed/y_train.json", )


# np.save( "../files/muhammed/x_train.json", x_train_muhammed )
# np.save("../files/muhammed/y_train.json",  y_train_muhammed )
# np.save( "../files/muhammed/x_test.json",x_test_muhammed )
# np.save( "../files/muhammed/y_test.json",y_test_muhammed )
# np.save( "../files/muhammed/classes.json",  classes )


In [ ]:
len(x_lstm_prossed_test) + len(x_lstm_prossed_train)